# Case study: Algorithmic trading
## Financial Data Analytics in Python
### Onur Akin **Sir**
#### 222221335
akin.sir@student.uni-halle.de <br>
https://onurakinsir.com

In [ ]:
# Backtesting
import yfinance as yf
from backtesting import Backtest, Strategy
import pandas as pd

In [2]:
symbol = yf.Ticker("BTC-USD")
price_data = symbol.history(interval="1m", period="7d")
display(price_data)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-07-09 00:00:00+00:00,30291.611328,30291.611328,30291.611328,30291.611328,0,0.0,0.0
2023-07-09 00:01:00+00:00,30297.105469,30297.105469,30297.105469,30297.105469,0,0.0,0.0
2023-07-09 00:02:00+00:00,30303.519531,30303.519531,30303.519531,30303.519531,0,0.0,0.0
2023-07-09 00:04:00+00:00,30304.685547,30304.685547,30304.685547,30304.685547,0,0.0,0.0
2023-07-09 00:05:00+00:00,30289.925781,30289.925781,30289.925781,30289.925781,0,0.0,0.0
...,...,...,...,...,...,...,...
2023-07-15 00:06:00+00:00,30311.550781,30311.550781,30311.550781,30311.550781,0,0.0,0.0
2023-07-15 00:08:00+00:00,30312.544922,30312.544922,30312.544922,30312.544922,0,0.0,0.0
2023-07-15 00:09:00+00:00,30294.546875,30294.546875,30294.546875,30294.546875,0,0.0,0.0


In [3]:
# Strategy

def MovingAverage(price, period):
    return pd.Series(price).rolling(period).mean()

class SMACross(Strategy):
    n_short = 45
    n_long = 65
    
    def init(self):
        close = self.data.Close
        self.ma_short = self.I(MovingAverage, close, self.n_short)
        self.ma_long = self.I(MovingAverage, close, self.n_long)

    def next(self):
        # We are going to check if we have crossover
        P_0 = self.data.Close[-1]
        P_1 = self.data.Close[-2]
        delta = 0.017
        
        if P_0 < (1 - delta) * P_1:
            if self.position:
                self.position.close()
                
        elif P_0 > (1 + delta) * P_1:
            if self.position:
                self.buy()
            else:
                self.buy()
                
        elif self.ma_short[-2] < self.ma_long[-2]:
            if self.ma_short[-1] > self.ma_long[-1]:
                self.buy()
            
        elif self.ma_short[-2] > self.ma_long[-2]:
            if self.ma_short[-1] < self.ma_long[-1]:
                if self.position:
                    self.position.close()

In [4]:
bt = Backtest(price_data, SMACross, cash=1000000000, commission=0, exclusive_orders=True)
output = bt.run()
bt.plot()

Row(id='1450', ...)

In [5]:
print(output)

Start                     2023-07-09 00:00...
End                       2023-07-15 00:11...
Duration                      6 days 00:11:00
Exposure Time [%]                   50.831914
Equity Final [$]            1034642956.785156
Equity Peak [$]             1040726952.103516
Return [%]                           3.464296
Buy & Hold Return [%]                0.015062
Return (Ann.) [%]                  358.255463
Volatility (Ann.) [%]              128.265179
Sharpe Ratio                         2.793084
Sortino Ratio                       64.618498
Calmar Ratio                       169.174438
Max. Drawdown [%]                   -2.117669
Avg. Drawdown [%]                   -0.570775
Max. Drawdown Duration        1 days 16:00:00
Avg. Drawdown Duration        0 days 08:15:00
# Trades                                   69
Win Rate [%]                        50.724638
Best Trade [%]                       1.360012
Worst Trade [%]                     -0.605307
Avg. Trade [%]                    

In [3]:
#  SMA(10) Strategy
# signal is the trader goes long position if P ≥ SMA(10)
# the trader goes short position if P ≤ SMA(10)
symbol = yf.Ticker("BTC-USD")
price_data = symbol.history(interval="1m", period="7d")
display(price_data)

def MovingAverage(price, period):
    return pd.Series(price).rolling(period).mean()

class SMACross(Strategy):
    n_short = 10
    
    def init(self):
        close = self.data.Close
        self.ma_short = self.I(MovingAverage, close, self.n_short)
    
    def next(self):
        P_0 = self.data.Close[-1]
        if P_0 >= self.ma_short:
            self.buy()
        elif P_0 <= self.ma_short:
            if self.position:
                self.position.close()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-07-11 00:00:00+00:00,30417.632812,30417.632812,30417.632812,30417.632812,0,0.0,0.0
2023-07-11 00:01:00+00:00,30423.056641,30423.056641,30423.056641,30423.056641,0,0.0,0.0
2023-07-11 00:03:00+00:00,30420.857422,30420.857422,30420.857422,30420.857422,4260864,0.0,0.0
2023-07-11 00:04:00+00:00,30423.626953,30423.626953,30423.626953,30423.626953,3532800,0.0,0.0
2023-07-11 00:06:00+00:00,30416.019531,30416.019531,30416.019531,30416.019531,10917888,0.0,0.0
...,...,...,...,...,...,...,...
2023-07-17 18:34:00+00:00,29883.330078,29883.330078,29883.330078,29883.330078,495616,0.0,0.0
2023-07-17 18:35:00+00:00,29821.023438,29821.023438,29821.023438,29821.023438,16356352,0.0,0.0
2023-07-17 18:36:00+00:00,29877.130859,29877.130859,29877.130859,29877.130859,74307584,0.0,0.0


In [4]:
bt = Backtest(price_data, SMACross, cash=1000000000, commission=0, exclusive_orders=True)
output = bt.run()
bt.plot()

Row(id='1429', ...)

In [5]:
print(output)

Start                     2023-07-11 00:00...
End                       2023-07-17 18:38...
Duration                      6 days 18:38:00
Exposure Time [%]                   56.742576
Equity Final [$]            1042554818.449219
Equity Peak [$]             1058183505.255859
Return [%]                           4.255482
Buy & Hold Return [%]               -1.570526
Return (Ann.) [%]                  260.978629
Volatility (Ann.) [%]              113.737534
Sharpe Ratio                         2.294569
Sortino Ratio                       13.379796
Calmar Ratio                        73.075949
Max. Drawdown [%]                   -3.571334
Avg. Drawdown [%]                   -0.207025
Max. Drawdown Duration        3 days 22:33:00
Avg. Drawdown Duration        0 days 02:32:00
# Trades                                 4388
Win Rate [%]                        50.592525
Best Trade [%]                       0.491553
Worst Trade [%]                     -0.546442
Avg. Trade [%]                    